In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib

In [ ]:
import os
import struct
import numpy as np
from utils import *



In [ ]:
import sklearn
from sklearn.manifold import TSNE
from time import time

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
from glow import thops
from glow import modules_origin
from glow import models_origin
# from glow import models
from glow.config import JsonConfig
import cv2
from CenterLoss import CenterLoss

In [ ]:
import importlib
reload(models_origin)
# reload(modules)

In [ ]:
# negative
path_total = '/home/hyshuai/workspace/dataset/puru'

dataset=[]
path_list = os.listdir(path_total)
path_list.sort()



for path_p in path_list:
        path_real = os.path.join(path_total,path_p)
        class_name = path_real
        # image_paths = get_image_paths(path_real)
        images = get_images_frompath_random_list_sort_wh(path_real,64,64,600)
        print(class_name," : ",len(images))
        dataset.append(ImageObject(class_name,images))
        

        
            

In [ ]:
dataset_negative_pre = []
dataset_train_positive =[]
dataset_test_positive = []
#袋鼠 狮子 河马 浣熊 熊猫

for img_obj in dataset:
    
    if img_obj.name.find('n01877812') == -1 and img_obj.name.find('n02129165') == -1 \
        and img_obj.name.find('n02398521') == -1 and img_obj.name.find('n02509815') == -1 \
        and img_obj.name.find('n02510455') == -1 :
        
        dataset_negative_pre.append(img_obj)
    else:
        dataset_train_positive.append(ImageObject(img_obj.name, img_obj.imgs[:5])) # 1-shot
        dataset_test_positive.append(ImageObject(img_obj.name, img_obj.imgs[5:]))
        print(img_obj.name)
        
    
    

In [ ]:
# positvie 
np.random.seed(66)
shufindex = np.random.permutation(len(dataset_negative_pre))

dataset_negative = np.array(dataset_negative_pre)[shufindex][:80]

print(len(dataset_negative[2].imgs),dataset_negative[2].name)


print(len(dataset_train_positive[2].imgs),dataset_train_positive[2].name)
print(len(dataset_test_positive[2].imgs))


In [ ]:
print(dataset_train_positive[1].name)
img_show_norm(np.transpose(dataset_train_positive[1].imgs[4],(1,2,0)))

In [ ]:

# label:

class_total_num = len(dataset_negative) + len(dataset_train_positive)
print("class_total = ",class_total_num)
np.random.seed(66) 
shufindex = np.random.permutation(class_total_num)
for index, value in enumerate(dataset_negative):
    value.label = shufindex[index]
    print(value.name,  value.label)

for index, value in enumerate(dataset_train_positive):
    value.label = shufindex[index+len(dataset_negative)]
    dataset_test_positive[index].label = shufindex[index+len(dataset_negative)]
    print(value.name, value.label)

In [ ]:
# test_class_index = [ 79, 77, 51, 60, 20]
test_class_index = [data.label for data in dataset_train_positive]
print(test_class_index)

In [ ]:

def get_next_single(dataset_train_positive,dataset_negative,p_size,n_size,shuffle = True):
    """
    final batch_size = p_size + n_size 
    every dataset_negative should have same imgs leng
    """
    #shuffle
    if shuffle:
        np.random.shuffle(dataset_train_positive)
        np.random.shuffle(dataset_negative)
        for data in dataset_negative:
            np.random.shuffle(data.imgs)
        for data in dataset_train_positive:
            np.random.shuffle(data.imgs)
    
    
    batch_negative = n_size 
#     print('batch_negative ',batch_negative)
    len_n_class = len(dataset_negative)
#     print('len_n_class ',len_n_class)
    len_p_class = len(dataset_train_positive)
#     print('len_p_class ',len_p_class)
    len_n_imgs = len(dataset_negative[0].imgs)
#     print('len_n_imgs ',len_n_imgs)
    len_p_imgs = len(dataset_train_positive[0].imgs)
    
    negative_number = 0
    
    
#     shuffindex = np.random.permutation(len_n)
# #     print(shuffindex)
#     np.random.shuffle(dataset_train_positive)
#     np.random.shuffle(dataset_negative)
    
    for data in dataset_negative:
        negative_number += len(data.imgs)
#         np.random.shuffle(data.imgs)
#     print('negative_number ',negative_number)
    batch_total = negative_number  // batch_negative
#     print('batch_total ',batch_total)
    i_n_class = 0
    i_n_img = 0
    i_p_class = 0
    i_p_img = 0
    
    
    
        
    
    for b in range(batch_total):
        train_x =[]
        train_y = []
        create_negative_imgs = 0
        while create_negative_imgs <= (batch_total * batch_negative) and create_negative_imgs < batch_negative:
            
            # while #如果一次循环没够，需要再次循环
            if i_n_class >= len_n_class:
                    i_n_class = 0
                    i_n_img +=1
                    if (i_n_img+1)  >len_n_imgs:
                        i_n_img = 0
#                     print('if i_n_class >= len_n_class i_n_class ',i_n_class)
#                     print('if i_n_class >= len_n_class i_n_img ',i_n_img)
            
            for i in range(i_n_class,len_n_class):
                    
#                 print('for i in range(i_n_class i_n_class ',i_n_class)
#                 print('for i in range(i_n_class i_n_img ',i_n_img)
                train_x += dataset_negative[i_n_class].imgs[i_n_img  : (i_n_img+1) ]
                train_y += [dataset_negative[i_n_class].label] 
                
                i_n_class+=1
                if i_n_class >= len_n_class:
                    i_n_class = 0
                    i_n_img +=1
                    if (i_n_img+1)  > len_n_imgs:
                        i_n_img = 0
#                     print('if i_n_class >= len_n_class i_n_class ',i_n_class)
#                     print('if i_n_class >= len_n_class i_n_img ',i_n_img)
            
                create_negative_imgs +=1
                if create_negative_imgs >= batch_negative:
#                     print("create_negative_imgs ok ",create_negative_imgs)
                    break
                    
                
        create_positive_imgs = 0    
        while create_positive_imgs < p_size:
            for i in range(i_p_class,len_p_class):
#                 print('for i in range(i_p_class     ',i_p_class)
#                 print('for i in range(i_p_img  ',i_p_img)
                
                train_x += dataset_train_positive[i_p_class].imgs[i_p_img:(i_p_img +1)]
                train_y += [dataset_train_positive[i_p_class].label]
                
                i_p_class+=1
                if i_p_class >= len_p_class:
                    i_p_class = 0
                    i_p_img +=1
                    if (i_p_img +1) > len_p_imgs:
                        i_p_img = 0
#                     print('if i_p_class >= len_p_class i_p_class ',i_p_class)
#                     print('if i_p_class >= len_p_class i_p_img ',i_p_img)
                
                create_positive_imgs+=1
                if create_positive_imgs >= p_size:
#                     print("create_positive_imgs ok ",create_positive_imgs)
                    break
        
        shuffindex_out = np.random.permutation(len(train_y))
#         print(train_x[0].shape)
        train_x = np.stack(train_x,axis = 0)
#         print('after concat : ',train_x[0].shape)
        train_x = train_x[shuffindex_out]
        train_y = np.array(train_y)[shuffindex_out]
        
        yield train_x,train_y  

In [ ]:

def get_next_perfect(dataset_train_positive,dataset_negative,p_size,n_size,shuffle = True):
    """
    final batch_size = p_size + n_size * 2
    every dataset_negative should have same imgs leng
    """
    #shuffle
    if shuffle:
        np.random.shuffle(dataset_train_positive)
        np.random.shuffle(dataset_negative)
        for data in dataset_negative:
            np.random.shuffle(data.imgs)
        for data in dataset_train_positive:
            np.random.shuffle(data.imgs)
    
    
    batch_negative = n_size * 2
#     print('batch_negative ',batch_negative)
    len_n_class = len(dataset_negative)
#     print('len_n_class ',len_n_class)
    len_p_class = len(dataset_train_positive)
#     print('len_p_class ',len_p_class)
    len_n_imgs = len(dataset_negative[0].imgs)
#     print('len_n_imgs ',len_n_imgs)
    len_p_imgs = len(dataset_train_positive[0].imgs)
    
    negative_number = 0
    
    
#     shuffindex = np.random.permutation(len_n)
# #     print(shuffindex)
#     np.random.shuffle(dataset_train_positive)
#     np.random.shuffle(dataset_negative)
    
    for data in dataset_negative:
        negative_number += len(data.imgs)
#         np.random.shuffle(data.imgs)
#     print('negative_number ',negative_number)
    batch_total = negative_number  // batch_negative
#     print('batch_total ',batch_total)
    i_n_class = 0
    i_n_img = 0
    i_p_class = 0
    i_p_img = 0
    
    
    
        
    
    for b in range(batch_total):
        train_x =[]
        train_y = []
        create_negative_imgs = 0
        while create_negative_imgs <= (batch_total * batch_negative) and create_negative_imgs < batch_negative:
            
            # while #如果一次循环没够，需要再次循环
            if i_n_class >= len_n_class:
                    i_n_class = 0
                    i_n_img +=1
                    if (i_n_img+1) *2 >len_n_imgs:
                        i_n_img = 0
#                     print('if i_n_class >= len_n_class i_n_class ',i_n_class)
#                     print('if i_n_class >= len_n_class i_n_img ',i_n_img)
            
            for i in range(i_n_class,len_n_class):
                    
#                 print('for i in range(i_n_class i_n_class ',i_n_class)
#                 print('for i in range(i_n_class i_n_img ',i_n_img)
                train_x += dataset_negative[i_n_class].imgs[i_n_img * 2 : (i_n_img+1) *2]
                train_y += [dataset_negative[i_n_class].label] * 2
                
                i_n_class+=1
                if i_n_class >= len_n_class:
                    i_n_class = 0
                    i_n_img +=1
                    if (i_n_img+1) *2 > len_n_imgs:
                        i_n_img = 0
#                     print('if i_n_class >= len_n_class i_n_class ',i_n_class)
#                     print('if i_n_class >= len_n_class i_n_img ',i_n_img)
            
                create_negative_imgs +=2
                if create_negative_imgs >= batch_negative:
#                     print("create_negative_imgs ok ",create_negative_imgs)
                    break
                    
                
        create_positive_imgs = 0    
        while create_positive_imgs < p_size:
            for i in range(i_p_class,len_p_class):
#                 print('for i in range(i_p_class     ',i_p_class)
#                 print('for i in range(i_p_img  ',i_p_img)
                
                train_x += dataset_train_positive[i_p_class].imgs[i_p_img:(i_p_img +1)]
                train_y += [dataset_train_positive[i_p_class].label]
                
                i_p_class+=1
                if i_p_class >= len_p_class:
                    i_p_class = 0
                    i_p_img +=1
                    if (i_p_img +1) > len_p_imgs:
                        i_p_img = 0
#                     print('if i_p_class >= len_p_class i_p_class ',i_p_class)
#                     print('if i_p_class >= len_p_class i_p_img ',i_p_img)
                
                create_positive_imgs+=1
                if create_positive_imgs >= p_size:
#                     print("create_positive_imgs ok ",create_positive_imgs)
                    break
        
        shuffindex_out = np.random.permutation(len(train_y))
#         print(train_x[0].shape)
        train_x = np.stack(train_x,axis = 0)
#         print('after concat : ',train_x[0].shape)
        train_x = train_x[shuffindex_out]
        train_y = np.array(train_y)[shuffindex_out]
        
        yield train_x,train_y  

In [ ]:
# dataaaaaaa = get_next_single(dataset_train_positive,dataset_negative,1,11)
# # x,y = dataaaaaaa.next()
# # print('x o shape ' ,x.shape,y.shape)
# for x ,y in dataaaaaaa:
#     print(y)
#     print('x o shape ' ,x.shape,y.shape)
#     print("one batch is over")
dataaaaaaa =None

In [ ]:
def getnext_test(dataset_test_positive,test_class_index):
    
    for data in dataset_test_positive:
        x = np.stack(data.imgs,axis =0)
        y = np.array([test_class_index.index(data.label)] * len(data.imgs))
        yield x,y

In [ ]:
# dataaaaaaa = getnext_test(dataset_test_positive,test_class_index)
# for x ,y in dataaaaaaa:
#     print(y)
#     print(x.shape,y.shape)

In [ ]:
def imgs_show_norm(imgs,row,col):
    fig,ax = plt.subplots(nrows=row,ncols=col,sharex=True,sharey=True)
    ax = ax.flatten()
    for i in range(row*col):
        img = imgs[i]
        min = np.min(img)
#         print('min = ',min)
        img = np.subtract(img,min) # 0->
        max = np.max(img)
#         print('max =',max)
        img = np.divide(img,max)
        
        ax[i].imshow(img,cmap='Greys', interpolation='nearest')
    
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    plt.tight_layout()
    plt.show()

In [ ]:
def img_show_norm(img,size=1):
    min_ = np.min(img)
#     print('min = ',min_)
    img = np.subtract(img,min_) # 0->
    max_ = np.amax(a = img,keepdims=False)
#     print('max =',max_)
    img = np.divide(img,max_)  # 0->1
#     print(img)
    plt.figure(figsize=(size,size))
    plt.imshow(img,interpolation='nearest')

In [ ]:
def img_show(img):
    plt.figure(figsize=(1,1))
    plt.imshow(img,interpolation='nearest')
    

In [ ]:
def imgs_show(imgs,row,col):
    fig,ax = plt.subplots(nrows=row,ncols=col,sharex=True,sharey=True)
    ax = ax.flatten()
    for i in range(row*col):
        img = imgs[i]
        
        ax[i].imshow(img,cmap='Greys', interpolation='nearest')
    
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    plt.tight_layout()
    plt.show()
    
    

In [ ]:
def compute_index_logical_equal(inputs,indexs):
    logic_result = (inputs==indexs[0])
    for value in indexs:
        logic_result = np.logical_or(inputs == value,logic_result)
    return logic_result


In [ ]:
# nb_digits = class_total_num
net_auto = models_origin.Glow(JsonConfig("hparams/celeba_imagenet.json"),L=4,K=32)

print(net_auto)
# centerloss = CenterLoss(class_total_num, 96)

In [ ]:

# net_auto.load_state_dict(torch.load('model_glow_imagenet_32k_5way_5shot_add_split_z_is_mean_10.pt'),strict=True)
# centerloss.load_state_dict(torch.load('model_center_imagenet_16k_5way_5shot_add_split_z_is_mean_center_0_02_8.pt'),strict=True)
# for s in net_auto.flow.output_shapes:
#     print(s)

In [ ]:

use_cuda = torch.cuda.is_available()
torch.manual_seed(666)
device = torch.device("cuda" if use_cuda else "cpu")



# criterion = nn.MSELoss()

net_auto.to(device)
net_auto.float()
# centerloss.to(device)
# centerloss.float()


In [ ]:
torch.cuda.empty_cache()

In [ ]:

center_loss_weight = 0.02
optimizer = optim.Adam(net_auto.parameters(),lr=0.001) # decoder don't weight_decay
# optimzer4center = optim.SGD(centerloss.parameters(), lr =0.5)

In [ ]:
#X_train_nor
# net_auto.set_z_add_random(True)
batch_size = 16
net_auto.train()
for epoch in range(30001):
    #train:
    
    datas =  get_next_single(dataset_train_positive,dataset_negative,0,16,shuffle=True)
#     datas = getnext(dataset_train_positive,dataset_negative)
    
    trainloss_g = 0
    trainloss_c = 0
    
    for x,y in datas:
        inputs ,lables = torch.from_numpy(x).float().to(device),\
            torch.from_numpy(y).long().to(device)
#         print(torch.sum(lables)) # = 1167
        
        y_onehot = torch.FloatTensor(batch_size, class_total_num).to(device)
        y_onehot.zero_()
        y_onehot.scatter_(1, lables.view(-1,1), 1)
#         print('y_onehot:', y_onehot)


       

        optimizer.zero_grad()
#         optimzer4center.zero_grad()
        
        z, det, y_logits= net_auto(x=inputs, y_onehot=y_onehot)
#         y_logits = net_auto.class_flow(z)
#         print("out size = ",z.size())
        
        loss_g = models_origin.Glow.loss_generative(det)
        loss_c = models_origin.Glow.loss_class(y_logits,lables)
#         loss_center = centerloss(lables, z.mean(2).mean(2))
        
#         print('%d  loss_g: %.4f, loss_c:%.4f ,loss_center: %.4f' % (epoch , loss_g.item(),loss_c.item(),loss_center.item()))
        print('%d  loss_g: %.4f, loss_c:%.4f' % (epoch , loss_g.item(),loss_c.item()))
        loss = loss_g + loss_c * 0.01 #+ center_loss_weight * loss_center
        loss.backward()
        torch.nn.utils.clip_grad_value_(net_auto.parameters(), 5)
        torch.nn.utils.clip_grad_norm_(net_auto.parameters(), 100)
        
        
        trainloss_g += loss_g.item()
        trainloss_c += loss_c.item()
        
        optimizer.step()
#         optimzer4center.step()
        
    print('%d  gloss: %.3f, class loss:%.3f' % (epoch ,  trainloss_g,trainloss_c))
    
    if epoch % 2 == 0 and epoch >0:
        
        torch.save(net_auto.state_dict(),'model_glow_imagenet_32k_5way_5shot_add_split_z_is_mean_no_p_'+str(epoch)+'.pt')
#         torch.save(centerloss.state_dict(),'model_center_imagenet_16k_5way_5shot_add_split_z_is_mean_center_0_02_'+str(epoch)+'.pt')
#24  loss_g: 3.3515, loss_c:2.6742
#24  loss_g: 3.3070, loss_c:2.8227
#24  loss_g: 3.3131, loss_c:2.9545    
# 35  loss_g: 3.5125, loss_c:1.9239
#29  loss_g: 3.4648, loss_c:2.5662
#29  loss_g: 3.3188, loss_c:2.9107
#29  loss_g: 3.4770, loss_c:1.8553


In [ ]:
# net_auto.set_z_add_random(True)
# net_auto.train()
epoch = 78
# torch.save(net_auto.state_dict(),'model_glow_imagenet_32k_5way_5shot_add_split_z_is_mean_'+str(epoch)+'.pt')
# torch.save(centerloss.state_dict(),'model_center_imagenet_16k_5way_5shot_add_split_z_is_mean_center_0_02_11.pt')

In [ ]:
# net_auto.set_z_add_random(False)
sample_p =  np.random.randint(5)  # 10 # 
sample_p_index =  np.random.randint(5)
sample_n =  np.random.randint(80)       # 669 # 292 #   500 #    669 # |
sample_index = np.random.randint(600)    # 13  # 1 #    # 13   #
print('sample_p = {},sample_n ={}, sample_index = {}'.format(sample_p,sample_n,sample_index))

posi = np.concatenate(( np.array(dataset_train_positive[sample_p].imgs[sample_p_index:sample_p_index+1]) , \
                       np.array(dataset_negative[sample_n].imgs[sample_index:sample_index+2])))
labl =  np.array([dataset_train_positive[sample_p].label, dataset_negative[sample_n].label,dataset_negative[sample_n].label])
print(dataset_train_positive[sample_p].name)
print(dataset_negative[sample_n].name)
print(labl)
inputs ,lables = torch.from_numpy(posi).float().to(device),\
            torch.from_numpy(labl).long().to(device)
        
y_onehot = torch.FloatTensor(3, class_total_num).to(device)
y_onehot.zero_()
y_onehot.scatter_(1, lables.view(-1,1), 1)
# print('y_onehot:', y_onehot)
# z, det, y_logits = net_auto(x=inputs, y_onehot=y_onehot)
z, det,y_lo = net_auto(x=inputs, y_onehot=y_onehot)
# y_logits = net_auto.class_flow(z)


obj = z[0] #+ 0.5 *(z[1] - z[2])
obj = obj.view(-1,*obj.size())
obj_oneshot = y_onehot[0].view(-1,class_total_num)
x_ = net_auto(z = obj,y_onehot = obj_oneshot,eps_std = 1e-8 ,reverse=True)
x_ = torch.clamp(x_,-1,1)
print(x_.size())

print("abs sum = ",torch.sum(torch.abs(x_[0] - inputs[0])))
# c = criterion(x_[0] , inputs[0])
# print("criterion sum   = ",c)
# c_self = criterion(inputs[0],ones_tensor)
# print("self criterion sum  = ",c_self)
# print("divider square = ", torch.sum(torch.mul(divider_z,divider_z)))
# print("l2  bi li   = ",c / c_self)


In [ ]:
# print(x_.data.size())
# img_show_norm(np.transpose(dataset_negative[1].imgs[5],(1,2,0)))
# img_show_norm(np.transpose(np.squeeze( x_.data.cpu().numpy()),(1,2,0)))
# img_show_norm(np.transpose(np.squeeze( inputs.data[0].cpu().numpy()),(1,2,0)))
# img_show_norm(np.transpose(np.squeeze( inputs.data[1].cpu().numpy()),(1,2,0)))
# img_show_norm(np.transpose(np.squeeze( inputs.data[2].cpu().numpy()),(1,2,0)))
x_ = None
inputs = None
z = None
det =None
y_lo = None
obj = None

In [ ]:
img_show_norm(np.transpose(np.squeeze( x_.data.cpu().numpy()),(1,2,0)))

In [ ]:
# net_auto.load_state_dict(torch.load('model_glow_ominst_all_bg_8k_20_way_5_shot_160.pt'))
net_auto.set_z_add_random(False)
use_cuda = torch.cuda.is_available()
torch.manual_seed(666)
device = torch.device("cuda" if use_cuda else "cpu")
net_auto.to(device)
net_auto.float()
net_auto.eval()

test_loss = 0
correct = 0
test_datas = getnext_test(dataset_test_positive,test_class_index)
for x ,y in test_datas:
        inputs, labels = Variable(torch.from_numpy(x).float()).cuda(), Variable(torch.from_numpy(y).long()).cuda()
#         print(labels)
        
        z, det = net_auto(x=inputs, y_onehot=None)
        classify = net_auto.class_flow(z)
        
        
        
#         _,classify = net_auto(inputs)
        test_loss += F.nll_loss(classify, labels, size_average=False).item() # sum up batch loss
#         print(classify)
        pred = classify.max(1,keepdim=True)[1]
        print(pred)
        
        correct += pred.eq(labels.view_as(pred)).sum().item()
#         print(correct)
print(correct)
print(correct * 1.0 / (19 * 20))
print(test_loss)

In [ ]:

# print(x_p.shape)
print(label_positive)
imgs_show(np.squeeze(x_positive),4,5)



In [ ]:
imgs_show(np.squeeze(images_observe_train_total[80:100]),4,5)

In [ ]:
imgs_show(np.squeeze(images_observe_train_total[0:20]),4,5)

In [ ]:
def get_real_mean_var():
    batch_size = 256
    net_auto.set_z_add_random(False)
    net_auto.eval()
    mean_std_list =[MeanVarObject(i) for i in range(class_total_num)]
    
    datas =  get_next_perfect(dataset_train_positive,dataset_negative,6,125,shuffle=False)
    
    for x,y in datas:
        inputs ,lables = torch.from_numpy(x).float().to(device),\
            torch.from_numpy(y).long().to(device)
#         print(torch.sum(lables)) # = 1167
        
        y_onehot = torch.FloatTensor(batch_size, class_total_num).to(device)
        y_onehot.zero_()
        y_onehot.scatter_(1, lables.view(-1,1), 1)
#         print('y_onehot:', y_onehot)
        optimizer.zero_grad()
        optimzer4center.zero_grad()
        
        z , _ = net_auto(x=inputs, y_onehot=y_onehot)
        z_real = z.detach().cpu().numpy()
        for i,label in enumerate(y) :
            mean_std_list[label].add_z(z_real[i])
    return mean_std_list

mean_std_obj_list = get_real_mean_var()

In [ ]:
# mean_std_list label : 0 ->984


mean_list = []
std_list =[]
z_total_list = []

for obj in mean_std_obj_list:
#     print(obj.label)
    mean_list.append(obj.get_mean())
    std_list.append(obj.get_std())
    z_total_list.append(obj.get_z_array())

mean_list  = np.array(mean_list)
std_list = np.array(std_list)
z_total_list = np.concatenate(np.array(z_total_list))


In [ ]:

print(z_total_list.shape)
mean_z = np.mean(z_total_list,axis=0)
std_z = np.std(z_total_list,axis=0)
color_z = z_total_list * 250
# print(mean_std_obj_list[500].get_mean()[2,:,:])
# print(mean_list.shape)
# print(mean_list[500,2,:,:])
# # print(mean_x[500,2,:,:])
# # print(mean_x[20,2,:,:])
# print(mean_std_obj_list[500].get_std()[2,:,:])
# print(std_list[500,2,:,:])




In [ ]:
def compute_distance_u_total(z_list,mean_z):
    # mean_std_obj_list[381].get_z_array()
    divider = np.subtract(z_list,mean_z) ** 2
#     print(divider.shape)
    return np.mean(divider,axis=0)

In [ ]:
# plt.figure(figsize=(2000,1),dpi=30)
# # plt.text(17527,100,'Pre')
# # my_y_ticks = np.arange(-0.1, 0.1, 0.01)
# # plt.yticks(my_y_ticks)
# d = 2
# w = 1
# h = 0
# print('std_z = ',std_z[d,w,h])
# print('mean = ',u_total_real[d,w,h])

# plt.scatter(x = z_total_list[:,d,w,h],y = np.ones_like(z_total_list[:,d,w,h]),s=75,c =color_z[:,d,w,h],  alpha=.5)
# plt.show()

In [ ]:
print('std_z = ',std_z[0:10])

In [ ]:
color = mean_list * 250
u_total_real = np.mean(mean_list,axis=0)
std_mean_real  = np.mean(np.square(std_list),axis=0)
ui_squ_mean_real  = np.mean(np.square(mean_list),axis=0)
u_total_sque_real = np.square(u_total_real)
std_squ_total = std_mean_real + ui_squ_mean_real - u_total_sque_real
std_total_real = np.sqrt(std_squ_total)

In [ ]:
print('mean = ',u_total_real[2,:,:])
print("  ",mean_list[20,2,:,:])

In [ ]:
# print('u_total =  ',u_total_real[2,:,:])
# print('std_mean =  ',np.sqrt(std_mean_real[:,0,0]))
# print('ui_squ_mean =  ',ui_squ_mean.shape)
# print('u_total_sque =  ',u_total_sque.shape)
# print('std_squ_total =  ',std_squ_total.shape)
d = 2
w = 1
h = 3
print('std_z = ',std_z[d,w,h])
print('mean = ',u_total_real[d,w,h])
print('std_list = ',std_list[100:200,d,w,h])
# print('mean list                = \n',mean_list[25:80,d,w,h])


In [ ]:
plt.figure(figsize=(300,2))

# my_y_ticks = np.arange(-0.1, 0.1, 0.01)
# plt.yticks(my_y_ticks)


print('std = ',std_total_real[d,w,h])
print('mean = ',u_total_real[d,w,h])
plt.scatter(x = mean_list[:,d,w,h],y = np.ones_like(mean_list[:,d,w,h]),s=75,c =color[:,d,w,h],  alpha=.5)
plt.show()

In [ ]:
lables_obser_bias = [241 ,188, 635, 270, 774, 790, 772, 376, 201 , 26, 453, 785 ,875 ,634 ,527 ,718 ,299 ,  6 ,679, 373]
for index ,value in enumerate(lables_obser_bias):
    print(str(index+1)," scale = ",np.sum(np.sort(np.reshape(std_list[value,:,:,:],64*4*4))[-50:]) / np.sum(std_list[value,:,:,:]) )

In [ ]:
lables_obser_bias = [543, 119 ,482 ,541, 806, 602, 211, 757, 600, 969, 942, 107, 752, 729, 698, 153 ,466 ,597,578 ,604]
for index ,value in enumerate(lables_obser_bias):
    print(str(index+1)," scale = ",np.sum(np.sort(np.reshape(std_list[value,:,:,:],64*4*4))[-50:]) / np.sum(std_list[value,:,:,:]) )

In [ ]:
lables_obser_bias = [344 ,539, 446, 566 ,431, 680, 161, 164, 336 ,980, 132 , 38, 169 ,660, 795 ,286 ,337 ,939,710, 973]
for index ,value in enumerate(lables_obser_bias):
    print(str(index+1)," scale = ",np.sum(np.sort(np.reshape(std_list[value,:,:,:],64*4*4))[-50:]) / np.sum(std_list[value,:,:,:]) )
    

In [ ]:
print("---------------------------------")
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[847,:,:,:],64*4*4))[-40:]) / np.sum(std_list[847,:,:,:]) )
print("118 scale = ",np.sum(np.sort(np.reshape(std_list[118,:,:,:],64*4*4))[-40:]) / np.sum(std_list[118,:,:,:]) )
print("364 scale = ",np.sum(np.sort(np.reshape(std_list[364,:,:,:],64*4*4))[-40:]) / np.sum(std_list[364,:,:,:]) )
print("239 scale = ",np.sum(np.sort(np.reshape(std_list[239,:,:,:],64*4*4))[-40:]) / np.sum(std_list[239,:,:,:]) )
print("76 scale = ",np.sum(np.sort(np.reshape(std_list[76,:,:,:],64*4*4))[-40:]) / np.sum(std_list[76,:,:,:]) )
print("---------------------------------")
print("936 scale = ",np.sum(np.sort(np.reshape(std_list[936,:,:,:],64*4*4))[-40:]) / np.sum(std_list[936,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[621,:,:,:],64*4*4))[-40:]) / np.sum(std_list[621,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[275,:,:,:],64*4*4))[-40:]) / np.sum(std_list[275,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[121,:,:,:],64*4*4))[-40:]) / np.sum(std_list[121,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[312,:,:,:],64*4*4))[-40:]) / np.sum(std_list[312,:,:,:]) )
print("---------------------------------")
print("---------------------------------")
print("---------------------------------")
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[859,:,:,:],64*4*4))[-40:]) / np.sum(std_list[859,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[389,:,:,:],64*4*4))[-40:]) / np.sum(std_list[389,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[536,:,:,:],64*4*4))[-40:]) / np.sum(std_list[536,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[730,:,:,:],64*4*4))[-40:]) / np.sum(std_list[730,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[332,:,:,:],64*4*4))[-40:]) / np.sum(std_list[332,:,:,:]) )
print("---------------------------------")
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[760,:,:,:],64*4*4))[-40:]) / np.sum(std_list[760,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[397,:,:,:],64*4*4))[-40:]) / np.sum(std_list[397,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[463,:,:,:],64*4*4))[-40:]) / np.sum(std_list[463,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[750,:,:,:],64*4*4))[-40:]) / np.sum(std_list[750,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[803,:,:,:],64*4*4))[-40:]) / np.sum(std_list[803,:,:,:]) )

print("---------------------------------")
print("---------------------------------")
print("---------------------------------")
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[886,:,:,:],64*4*4))[-40:]) / np.sum(std_list[886,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[882,:,:,:],64*4*4))[-40:]) / np.sum(std_list[882,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[215,:,:,:],64*4*4))[-40:]) / np.sum(std_list[215,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[430,:,:,:],64*4*4))[-40:]) / np.sum(std_list[430,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[861,:,:,:],64*4*4))[-40:]) / np.sum(std_list[861,:,:,:]) )
print("---------------------------------")
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[812,:,:,:],64*4*4))[-40:]) / np.sum(std_list[812,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[623,:,:,:],64*4*4))[-40:]) / np.sum(std_list[623,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[381,:,:,:],64*4*4))[-40:]) / np.sum(std_list[381,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[338,:,:,:],64*4*4))[-40:]) / np.sum(std_list[338,:,:,:]) )
print("847 scale = ",np.sum(np.sort(np.reshape(std_list[840,:,:,:],64*4*4))[-40:]) / np.sum(std_list[840,:,:,:]) )



# print("std fuza = ",np.sum(std_list[847,:,:,:]))
# print("std fuza = ",np.sum(std_list[118,:,:,:]))
# print("std fuza = ",np.sum(std_list[364,:,:,:]))
# print("std fuza = ",np.sum(std_list[239,:,:,:]))
# print("std fuza = ",np.sum(std_list[76,:,:,:]))
# print("---------------------------------")
# print("std fuza = ",np.sum(std_list[936,:,:,:]))
# print("std fuza = ",np.sum(std_list[621,:,:,:]))
# print("std fuza = ",np.sum(std_list[275 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[121 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[312 ,:,:,:]))
# print("---------------------------------")
# print("---------------------------------")
# print("---------------------------------")
# print("std fuza = ",np.sum(std_list[859,:,:,:]))
# print("std fuza = ",np.sum(std_list[389,:,:,:]))
# print("std fuza = ",np.sum(std_list[536 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[730 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[332 ,:,:,:]))
# print("---------------------------------")
# print("std fuza = ",np.sum(std_list[760,:,:,:]))
# print("std fuza = ",np.sum(std_list[397,:,:,:]))
# print("std fuza = ",np.sum(std_list[463 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[750 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[803 ,:,:,:]))
# print("---------------------------------")
# print("---------------------------------")
# print("---------------------------------")
# print("std fuza = ",np.sum(std_list[886,:,:,:]))
# print("std fuza = ",np.sum(std_list[882,:,:,:]))
# print("std fuza = ",np.sum(std_list[215 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[430 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[861 ,:,:,:]))
# print("---------------------------------")
# print("std fuza = ",np.sum(std_list[812,:,:,:]))
# print("std fuza = ",np.sum(std_list[623,:,:,:]))
# print("std fuza = ",np.sum(std_list[381 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[338 ,:,:,:]))
# print("std fuza = ",np.sum(std_list[840 ,:,:,:]))




In [ ]:
print("---------------------------------")
print("std fuza = ",np.sum(std_list[849,:,:,:]))
print("std fuza = ",np.sum(std_list[5,:,:,:]))
print("std fuza = ",np.sum(std_list[101,:,:,:]))
print("std fuza = ",np.sum(std_list[590,:,:,:]))
print("std fuza = ",np.sum(std_list[235,:,:,:]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[135,:,:,:]))
print("std fuza = ",np.sum(std_list[214,:,:,:]))
print("std fuza = ",np.sum(std_list[465 ,:,:,:]))
print("std fuza = ",np.sum(std_list[411 ,:,:,:]))
print("std fuza = ",np.sum(std_list[901 ,:,:,:]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[54,:,:,:]))
print("std fuza = ",np.sum(std_list[977,:,:,:]))
print("std fuza = ",np.sum(std_list[580 ,:,:,:]))
print("std fuza = ",np.sum(std_list[764 ,:,:,:]))
print("std fuza = ",np.sum(std_list[403 ,:,:,:]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[519,:,:,:]))
print("std fuza = ",np.sum(std_list[864,:,:,:]))
print("std fuza = ",np.sum(std_list[916 ,:,:,:]))
print("std fuza = ",np.sum(std_list[675 ,:,:,:]))
print("std fuza = ",np.sum(std_list[964 ,:,:,:]))


print(np.sum(np.sort(np.reshape(std_list[977,:,:,:],64*4*4))[-40:]))
print(np.sum(np.sort(np.reshape(std_list[580,:,:,:],64*4*4))[-40:]))

In [ ]:
print("---------------------------------")
print("std fuza = ",np.sum(std_list[220,:,:,:]))
print("std fuza = ",np.sum(std_list[48,:,:,:]))
print("std fuza = ",np.sum(std_list[379,:,:,:]))
print("std fuza = ",np.sum(std_list[294,:,:,:]))
print("std fuza = ",np.sum(std_list[18,:,:,:]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[674,:,:,:]))
print("std fuza = ",np.sum(std_list[97,:,:,:]))
print("std fuza = ",np.sum(std_list[348 ,:,:,:]))
print("std fuza = ",np.sum(std_list[65 ,:,:,:]))
print("std fuza = ",np.sum(std_list[831 ,:,:,:]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[547,:,:,:]))
print("std fuza = ",np.sum(std_list[844,:,:,:]))
print("std fuza = ",np.sum(std_list[236 ,:,:,:]))
print("std fuza = ",np.sum(std_list[172 ,:,:,:]))
print("std fuza = ",np.sum(std_list[424 ,:,:,:]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[657,:,:,:]))
print("std fuza = ",np.sum(std_list[46,:,:,:]))
print("std fuza = ",np.sum(std_list[3 ,:,:,:]))
print("std fuza = ",np.sum(std_list[255 ,:,:,:]))
print("std fuza = ",np.sum(std_list[470 ,:,:,:]))


print(np.sum(np.sort(np.reshape(std_list[46,:,:,:],64*4*4))[:-100]))
print(np.sum(np.sort(np.reshape(std_list[3,:,:,:],64*4*4))[:-100]))

In [ ]:
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[220].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[48].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[379].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[294].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[18].get_z_array(),mean_z)))
print("---------------------------------")

print('distance_u_simple  = ',np.sum(compute_distance_u_total(mean_std_obj_list[674].get_z_array(),mean_z)))
print('distance_u_simple  = ',np.sum(compute_distance_u_total(mean_std_obj_list[97].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[348].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[65].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[831].get_z_array(),mean_z)))
print("---------------------------------")
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[547].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[844].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[236].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[172].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[424].get_z_array(),mean_z)))
print("---------------------------------")
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[657].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[46].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[3].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[255].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[470].get_z_array(),mean_z)))

In [ ]:

# distance_u_simple = compute_distance_u_total(mean_std_obj_list[simple].get_z_array(),mean_z)

print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[159].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[687].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[486].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[843].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[27].get_z_array(),mean_z)))
print("---------------------------------")

print('distance_u_simple  = ',np.sum(compute_distance_u_total(mean_std_obj_list[253].get_z_array(),mean_z)))
print('distance_u_simple  = ',np.sum(compute_distance_u_total(mean_std_obj_list[909].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[295].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[912].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[714].get_z_array(),mean_z)))
print("---------------------------------")
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[307].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[661].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[858].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[79].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[717].get_z_array(),mean_z)))
print("---------------------------------")
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[122].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[51].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[631].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[828].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[20].get_z_array(),mean_z)))


# print("---------------------------------")
# print("std jiandan = ",np.sum(std_list[761,:,:,:]))
# print("std complex = ",np.sum(std_list[761,:,:,:]))

In [ ]:
print("---------------------------------")
print("std fuza = ",np.sum(std_list[159,:,:,:]))
print("std fuza = ",np.sum(std_list[687,:,:,:]))
print("std fuza = ",np.sum(std_list[486,:,:,:]))
print("std fuza = ",np.sum(std_list[843,:,:,:]))
print("std fuza = ",np.sum(std_list[27,:,:,:]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[253,:,:,:]))
print("std fuza = ",np.sum(std_list[909,:,:,:]))
print("std fuza = ",np.sum(std_list[295 ,:,:,:]))
print("std fuza = ",np.sum(std_list[912 ,:,:,:]))
print("std fuza = ",np.sum(std_list[714 ,:,:,:]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[307,:,:,:]))
print("std fuza = ",np.sum(std_list[661,:,:,:]))
print("std fuza = ",np.sum(std_list[858 ,:,:,:]))
print("std fuza = ",np.sum(std_list[79 ,:,:,:]))
print("std fuza = ",np.sum(std_list[717 ,:,:,:]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[122,:,:,:]))
print("std fuza = ",np.sum(std_list[51,:,:,:]))
print("std fuza = ",np.sum(std_list[631 ,:,:,:]))
print("std fuza = ",np.sum(std_list[828 ,:,:,:]))
print("std fuza = ",np.sum(std_list[20 ,:,:,:]))


print(np.sum(np.sort(np.reshape(std_list[79,:,:,:],64*4*4))[-10:]))
print(np.sum(np.sort(np.reshape(std_list[122,:,:,:],64*4*4))[-10:]))
print(np.sum(np.sort(np.reshape(std_list[486,:,:,:],64*4*4))[-10:]))

In [ ]:
complexs = 239
simple = 381
print("u_total = ",u_total_real[2,:,:])
print("std_real = ",std_total_real[2,:,:])
print("---------------------------------")
# print("fuza = ",mean_list[239,2,:,:])
# print("jiandan = ",mean_list[381,2,:,:])

print("---------------------------------")
# distance_u_complex = compute_distance_u_total(mean_std_obj_list[complexs].get_z_array(),mean_z)
print('distance_u_complex = ',np.sum(compute_distance_u_total(mean_std_obj_list[847].get_z_array(),mean_z)))
print('distance_u_complex = ',np.sum(compute_distance_u_total(mean_std_obj_list[118].get_z_array(),mean_z)))
print('distance_u_complex = ',np.sum(compute_distance_u_total(mean_std_obj_list[239].get_z_array(),mean_z)))
print('distance_u_complex = ',np.sum(compute_distance_u_total(mean_std_obj_list[76].get_z_array(),mean_z)))
print('distance_u_complex = ',np.sum(compute_distance_u_total(mean_std_obj_list[936].get_z_array(),mean_z)))
print('distance_u_complex = ',np.sum(compute_distance_u_total(mean_std_obj_list[621].get_z_array(),mean_z)))
print('distance_u_complex = ',np.sum(compute_distance_u_total(mean_std_obj_list[891].get_z_array(),mean_z)))
print('distance_u_complex = ',np.sum(compute_distance_u_total(mean_std_obj_list[586].get_z_array(),mean_z)))
print("---------------------------------")
# distance_u_simple = compute_distance_u_total(mean_std_obj_list[simple].get_z_array(),mean_z)
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[886].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[882].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[430].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[812].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[623].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[381].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[579].get_z_array(),mean_z)))
print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[559].get_z_array(),mean_z)))
# std_fuza = np.sort(np.reshape(std_list[239,:,:,:],16*4*4))
# std_jiandan = np.sort(np.reshape(std_list[381,:,:,:],16*4*4))
# print("std fuza = ",np.sum(std_fuza[100:150]) )
# print("std jiandan = ",np.sum(std_jiandan[100:150]))
print("---------------------------------")
print("std fuza = ",np.sum(std_list[847,:,:,:]))
print("std fuza = ",np.sum(std_list[118,:,:,:]))
print("std fuza = ",np.sum(std_list[239,:,:,:]))
print("std fuza = ",np.sum(std_list[76,:,:,:]))
print("std fuza = ",np.sum(std_list[936,:,:,:]))
print("std fuza = ",np.sum(std_list[621,:,:,:]))
print("std fuza = ",np.sum(std_list[621,:,:,:]))
print("std fuza = ",np.sum(std_list[366 ,:,:,:]))
print("std fuza = ",np.sum(std_list[292 ,:,:,:]))
print("std fuza = ",np.sum(std_list[479 ,:,:,:]))
print("std fuza = ",np.sum(std_list[955 ,:,:,:]))


# print("---------------------------------")
# print("std fuza = ",np.sum(std_list[661,:,:,:]))
# print("std fuza = ",np.sum(std_list[858,:,:,:]))
# print("std fuza = ",np.sum(std_list[79,:,:,:]))
# print("std fuza = ",np.sum(std_list[159,:,:,:]))
# print("---------------------------------")
print("std jiandan = ",np.sum(std_list[886,:,:,:]))
print("std jiandan = ",np.sum(std_list[882,:,:,:]))
print("std jiandan = ",np.sum(std_list[430,:,:,:]))
print("std jiandan = ",np.sum(std_list[812,:,:,:]))
print("std jiandan = ",np.sum(std_list[623,:,:,:]))
print("std jiandan = ",np.sum(std_list[381,:,:,:]))


print("---------------------------------")
print("std jiandan = ",np.sum(std_list[761,:,:,:]))
print("std jiandan = ",np.sum(std_list[589 ,:,:,:]))
print("std jiandan = ",np.sum(std_list[572 ,:,:,:]))
print("std jiandan = ",np.sum(std_list[129 ,:,:,:]))


# print("---------------------------------")
# print("std jiandan = ",np.sum(std_list[979,:,:,:]))
# print('distance_u_simple = ',np.sum(compute_distance_u_total(mean_std_obj_list[979].get_z_array(),mean_z)))

# mean_std_obj_list[979].get_z_array()[12:14]


In [ ]:
batch_size = len(label_observe_total)
lables = np.array([index for index in range(class_total_num)])
lables = torch.from_numpy(lables).long().to(device)
print("get_log_mean_var ,labels = ", lables)
print(lables[-1])
y_onehot = torch.FloatTensor(batch_size, class_total_num).cuda()
y_onehot.zero_()
y_onehot.scatter_(1, lables.view(-1, 1), 1)

mean,var = net_auto.get_log_mean_var(batch_size,y_onehot)

In [ ]:
std = torch.exp(var.cpu()).detach().numpy()
mean_x = mean.cpu().detach().numpy()
# mean_y = np.ones_like(mean_x)
color = (mean_x + 1 ) * 150

u_total = np.mean(mean_x,axis=0)
print('u_total =  ',u_total[:,0,0])
std_mean = np.mean(np.square(std),axis=0)
print('std_mean =  ',np.sqrt(std_mean[:,0,0]))
ui_squ_mean = np.mean(np.square(mean_x),axis=0)
print('ui_squ_mean =  ',ui_squ_mean.shape)
u_total_sque = np.square(u_total)
print('u_total_sque =  ',u_total_sque.shape)
std_squ_total = std_mean + ui_squ_mean - u_total_sque
std_total = np.sqrt(std_squ_total)
print('std_squ_total =  ',std_squ_total.shape)
print('std__total = ',std_total[2,:,:])
print('std__total = ',std_total_real[2,:,:])

In [ ]:
plt.figure(figsize=(200,5))

# my_y_ticks = np.arange(-0.1, 0.1, 0.01)
# plt.yticks(my_y_ticks)

plt.scatter(x = mean_x[:,9,0,0],y = np.ones_like(mean_x[:,9,0,0]),s=75,c =color[:,9,0,0],  alpha=.5)
plt.show()

In [ ]:
criterion = nn.MSELoss(reduction='sum')
ones_tensor = torch.ones((1, 32, 32)).float().cuda()
std_total_gpu = torch.from_numpy(std_total_real).cuda()
from scipy import misc
import time

In [ ]:
vec1 = mean_list[122]
vec2 = mean_list[470]
print(np.sqrt(np.sum(np.square( vec1 - vec2 ))))
print(float(np.sum(vec1*vec2)) / np.linalg.norm(vec1)*np.linalg.norm(vec2))
print("----------------------------------")
vec1 = mean_list[122]
vec2 = mean_list[101]
print(np.sqrt(np.sum(np.square( vec1 - vec2 ))))
print(float(np.sum(vec1*vec2)) / np.linalg.norm(vec1)*np.linalg.norm(vec2))
print("----------------------------------")
vec1 = mean_list[122]
vec2 = mean_list[590]
print(np.sqrt(np.sum(np.square( vec1 - vec2 ))))
print(float(np.sum(vec1*vec2)) / np.linalg.norm(vec1)*np.linalg.norm(vec2))



In [ ]:

criterion = nn.MSELoss(reduction='sum')
ones_tensor = torch.ones((3, 128, 128)).float().cuda()
from scipy import misc
import time

In [ ]:
# print(np.sqrt(np.sum(np.square( mean_div ))))
# mean_bi = np.clip( abs(mean_div / (std_total_real )),0,1)
# print(mean_bi[0:5])


# # mean_scale  = torch.from_numpy(np.cos(mean_bi)).cuda()
# mean_scale  = torch.reciprocal(torch.exp(torch.from_numpy(mean_bi).cuda()))
# print(mean_scale[0:5])
# nagi_div_bi_std = abs(z[2] - z[1]) / torch.from_numpy(std_list[lable_n]).cuda()
# print(nagi_div_bi_std[0:5])
# n_jian_chu_std = torch.clamp(2 / (nagi_div_bi_std + 1e-8),0,1.5)
# print(n_jian_chu_std[0:5])

In [ ]:

# print(z[0,0,:,:])
# print(z[1,0,:,:])
# print(z[2,0,:,:])
# divi = z[1]-z[0]
# print(divi[0])

# std_scale = 1.0 / np.exp(abs(mean_list[labl[0]] - mean_list[labl[1]]) / std_total_real ) * 0.5
# print(std_scale) std_total_gpu 可能非常小，造成divide非常大
# divide = torch.div(torch.abs((z[1]- z[0])), std_total_gpu * 0.5 ) 
# # scale = torch.cos(torch.clamp(divide,0,1.57))

# scale = torch.reciprocal(torch.exp(divide))* 0.4 

# # scale = torch.abs(torch.div(std_total_gpu * 0.3, torch.add(torch.sub(z[1],z[0]),1e-4)))
# # scale = torch.clamp(scale,0,1)
# print(scale)
# # scale = 0.39
# divider_z = scale * (z[2] - z[1]) 

# obj = z[0]+ divider_z

divide = torch.div(torch.abs(torch.sub(z[1],z[0])), std_total_gpu ) 
# scale = torch.cos(torch.clamp(divide,0,1.57))

scale = torch.reciprocal(torch.exp(divide)) 
# scale = torch.abs(torch.div(std_total_gpu * 0.3, torch.add(torch.sub(z[1],z[0]),1e-4)))
# scale = torch.clamp(scale,0,1)
# print(scale > 1e-1)
# scale = 0.39
divider_z = scale * (z[2] - z[1])
obj = z[0]+ divider_z


# obj = z[0]
# obj[0:30] = obj[0:30]+ divider_z[0:30]
# d = 0
# w = 0
# h = 0
# print('obj after divider = ' ,obj[d,w,h])
# obj[d,w,h] = obj[d,w,h] + 50 * (z[2,d,w,h] - z[1,d,w,h])
# print('obj after compute mean = ' ,obj[d,w,h])



# print('obj mean = ',mean_x[-7,:,0,0])
# print('std_squ_total = ',std_squ_total[:,0,0])

# print(obj.size())
obj = obj.view(-1,*obj.size())
# print(obj.size())
obj_oneshot = y_onehot[0].view(-1,class_total_num)



In [ ]:
x_ = net_auto(z = obj,y_onehot = obj_oneshot,eps_std = 1e-8 ,reverse=True)

x_ = torch.clamp(x_,-1,1)
print(x_.size())

print("abs sum = ",torch.sum(torch.abs(x_[0] - inputs[0])))
# print("abs sum = ",torch.sum(x_[0]- inputs[0]))
c = criterion(x_[0] , inputs[0])
print("criterion sum   = ",c)

c_self = criterion(inputs[0],ones_tensor)
print("self criterion sum  = ",c_self)
# dd =  torch.sum(torch.mul(x_[0],inputs[0]))
# print("  sum  = ",  dd )
# m1 = torch.norm(x_[0])
# print("  sum  = ",  m1 )
# m2 = torch.norm(inputs[0])
# print("  sum  = ",  m2 )

# l2_self = torch.sum(np.square(x_[0] - inputs[0]))
# print("l2_self sum  = ",l2_self)
print("divider square = ", torch.sum(torch.mul(divider_z,divider_z)))
print("l2  bi li   = ",c / c_self)
# print("cos distance   = ",dd  / (m1 * m2) )
print(str(sample_n)," scale = ",np.sum(np.sort(np.reshape(std_list[sample_n,:,:,:],64*4*4))[-50:]) / np.sum(std_list[sample_n,:,:,:]) )

In [ ]:
img_show(np.squeeze(x_.data))
img_show(np.squeeze(inputs.data[0]))

In [ ]:
imgs_show(np.squeeze(np.concatenate((x_.data,inputs.data)) ),1,4)

In [ ]:
imgs_show(np.squeeze(np.concatenate((x_.data,inputs.data)) ),1,4)

In [ ]:
imgs_show(np.squeeze(np.concatenate((x_.data,inputs.data)) ),1,4)

In [ ]:
imgs_show(np.squeeze(np.concatenate((x_.data,inputs.data)) ),1,4)

In [ ]:
dim_i = 10
print('mean_x -7 = ',mean_x[sample_p,:dim_i,0,0])
print('mean_x 100 = ',mean_x[100,:dim_i,0,0])
print('std -7 = ',std[sample_p,:dim_i,0,0])
print('std 100 = ',std[100,:dim_i,0,0])


print('u_total= ',u_total[:dim_i,0,0])
distance = mean_x[-7,:dim_i,0,0] -u_total[:dim_i,0,0]

print('sample mean distance mean_total = ', distance)
print('max =',np.max(distance))
print('min =',np.min(distance))

print('std__total = ',std_total)

In [ ]:
obj = z[0]

scale = 1.0
# print('scale = ', scale)
start_index = np.random.randint(0,44)
# start_index = 0
end_index = start_index + 20
print('scale = {},start_index ={}, end_index = {}'.format(scale,start_index,end_index))
# print('----------------------2-------------------------------')
# max_std_index = torch.argmax(std_[-7,start_index:end_index,0,0])
# print(max_std_index)
# print(std_[-7,start_index:end_index,0,0][max_std_index])
# print(torch.max(std_[-7,start_index:end_index,0,0]))
# divider =   scale * (z[1][start_index:end_index][max_std_index] - z[2][start_index:end_index][max_std_index])
# distance_1 = obj[start_index:end_index] +  divider - mean_[-7,start_index:end_index,:,:]












# slic_index = np.random.permutation(64)[:20]
# print(slic_index) 

divider =   z[1][start_index:end_index] - z[2][start_index:end_index]
distance_1 = obj[start_index:end_index] +  divider - mean_[sample_p,start_index:end_index,:,:]
# print('distance_1  = ' , torch.abs(distance_1))
distance =    distance_1  # / std_[sample_p,start_index:end_index,:,:]  #mean 取对应label
div_max_dim = flat_index_to_tensor(torch.argmax(torch.abs(distance)))
c,w,h = div_max_dim
print('----------------------1-------------------------------')

print('distance_max_dim  = ' , div_max_dim)
print('distance_ max   = ' , distance[div_max_dim])
print('distance__max  = ' , torch.max(torch.abs(distance)))
print('----------------------2-------------------------------')
print('std  = ' , std_[sample_p,c,w,h ])
print('mean_a1 = ',mean_[sample_p,c,w,h ])
u_total_t = torch.from_numpy(u_total).cuda()
print('mean_total  = ',u_total[c,w,h ])
print('a1  = ',obj[c,w,h])
print('----------------------3-------------------------------')
s_max = 1
c,w,h = div_max_dim
print(c,w,h)

a_u_distance_p = mean_[sample_p,c,w,h ] - obj[c,w,h] + s_max * std_[sample_p, c,w,h]
            


scale_real_p  =  a_u_distance_p / divider[c,w,h]
print(scale_real_p)
a_u_distance_n = mean_[sample_p, c,w,h ] - obj[c,w,h] - s_max * std_[sample_p,  c,w,h]
            
scale_real_n  =  a_u_distance_n / divider[c,w,h]
print(scale_real_n)


if torch.abs(obj[div_max_dim] + scale_real_p * divider[div_max_dim] - u_total_t[div_max_dim]) < \
        torch.abs(obj[div_max_dim] + scale_real_n * divider[div_max_dim]  - u_total_t[div_max_dim]) :
    scale_real = scale_real_p
else :
    scale_real = scale_real_n
# if scale_real < -0.5 :
#     scale_real = -0.5
# elif scale_real > 0.5:
#     scale_real = 0.5
if scale_real < 0.2 and scale_real > 0:
    scale_real = 0.2
elif scale_real >  -0.2 and scale_real < 0:
    scale_real = -0.2


# scale_real = -0.3

# if torch.abs(scale_real_p) < torch.abs(scale_real_n) :
#     scale_real = scale_real_p
# else :
#     scale_real = scale_real_n
# scale_real = -0.3
print('---------------------- 4 -------------------------------')
obj[start_index:end_index] = obj[start_index:end_index] + scale_real * divider
print('scale_real  = ',scale_real)
print('after scale_real   = ',obj[div_max_dim])

# obj[start_index:end_index] = z[0] + scale_real_n * divider
# print('after scale_real_n   = ',obj[div_max_dim])

obj = obj.view(-1,*obj.size())
obj_oneshot = y_onehot[0].view(-1,class_total_num)